#### The below code works but it is computationally more costly

In [1]:
# from ismn.interface import ISMN_Interface
# import matplotlib.pyplot as plt
# %matplotlib inline
# import pandas as pd
# import os
# import re
# import numpy as np
# import xarray as xr
# import geopandas as gpd
# from shapely.geometry import Point
# from datetime import datetime
# import warnings
# warnings.filterwarnings('ignore')

### Provide path for data (.zip)
- Downloaded from ISMN Website

In [2]:
# path = r'data\oceania\Data_separate_files_header_20160614_20250614_11438_viBk_20250614.zip'
# ismn_data = ISMN_Interface(path, parallel=True)
# network_list=[]
# for i in ismn_data.networks:
#     network_list.append(i)
# print('Data will be extracted for these station: ', network_list)

In [3]:
# from shapely.geometry import Point
# from src.ismn_utils import *

# export_format = 'csv'  #'geojson', 'shp', 'parquet', 'gpkg','csv'
# stat_operator=['mean','max','min','std', 'median']

# for stat in stat_operator:

#     for network in ismn_data.networks:
#         print(f'Processing network: {network}')

#         try:

#             sm = ismn_data[network].to_xarray(variable='soil_moisture')
            
#             if sm is None or len(sm.sensor)==0:
#                 print(f"No soil moisture data available for network: {network}. Skipping...\n{'-'*50}")
#                 continue

#             # Extract static parameters
#             static_df = get_static(sm)

#             # Extract time series soil moisture
#             ts_df=get_sm_time_series(sm)
#             ts_df=get_sm_time_series(sm, statistic=stat)

#             # Merge 
#             merged_df=pd.concat([static_df, ts_df], axis=1)
#             print('Dimention of static dataframe: ', static_df.shape)
#             print('Dimention of time series soil moisture dataframe: ',ts_df.shape)
#             print('Dimention of merged dataframe: ', merged_df.shape)

#             geometry = [Point(xy) for xy in zip(merged_df['longitude'], merged_df['latitude'])]
#             gdf = gpd.GeoDataFrame(merged_df, geometry=geometry, crs='EPSG:4326')


#             # Build output path without extension
#             output_path = os.path.join(os.path.split(path)[0], 'extracted_data',stat, f'{network}')
#             os.makedirs(os.path.dirname(output_path), exist_ok=True)

#             export_gdf(gdf, output_path, file_format=export_format)

#         except Exception as e:
#             print(f"Error processing network {network}:{e}\n{'-'*50}")

        


### Use this code instead 
- It is computationally efficient
- Run on batch 
- Gererates log

In [4]:
import os
import warnings
from ismn.interface import ISMN_Interface
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from src.ismn_utils import get_static, get_sm_time_series, export_gdf
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

In [5]:
# ------------------------- CONFIG -------------------------
data_root = 'data'  # Root folder containing continent subfolders
export_format = 'csv'
stat_operators = ['mean', 'max', 'min', 'std', 'median']

In [6]:
def process_network(ismn_data, network, stat, base_output_dir, raw_ts_cache, overwrite=False, log_list=None):
    stat_dir = os.path.join(base_output_dir, stat)
    os.makedirs(stat_dir, exist_ok=True)

    output_file = os.path.join(stat_dir, f"{network}")  # <-- No .csv extension
    if os.path.exists(os.path.join(output_file+'.'+export_format)) and not overwrite:
        msg = f"  ✔ Output exists for {network} - {stat}. Skipping."
        print(msg)
        log_list.append(msg)
        return "skipped"

    try:
        sm = ismn_data[network].to_xarray(variable='soil_moisture')
        if sm is None or len(sm.sensor) == 0:
            msg = f"  ✘ No soil moisture for {network}."
            print(msg)
            log_list.append(msg)
            return "failed"

        print(f"  ⏳ Processing {network} with {len(sm.sensor)} sensors...")

        # Cache xarray object once per network
        if network not in raw_ts_cache:
            raw_ts_cache[network] = sm

        static_df = get_static(sm)
        ts_df = get_sm_time_series(raw_ts_cache[network], statistic=stat)

        merged_df = pd.concat([static_df, ts_df], axis=1)
        geometry = [Point(xy) for xy in zip(merged_df['longitude'], merged_df['latitude'])]
        gdf = gpd.GeoDataFrame(merged_df, geometry=geometry, crs='EPSG:4326')

        export_gdf(gdf, output_file, file_format=export_format)

        msg = f"  ✓ Exported: {output_file}"
        print(msg)
        log_list.append(msg)
        return "success"

    except Exception as e:
        msg = f"  ⚠ Error with {network} - {stat}: {e}"
        print(msg)
        log_list.append(msg)
        return "failed"
    
def process_ismn_file(file_path, overwrite=False):
    print(f"\n📁 Processing file: {os.path.basename(file_path)}")

    log = []
    result_counter = {"success": 0, "skipped": 0, "failed": 0}

    try:
        ismn_data = ISMN_Interface(file_path, parallel=True)
    except Exception as e:
        msg = f"⚠ Failed to load {file_path}: {e}"
        print(msg)
        log.append(msg)
        return log, result_counter

    raw_ts_cache = {}
    base_output_dir = os.path.join(os.path.dirname(file_path), 'extracted_data')

    for stat in stat_operators:
        print(f"\n🔄 Stat: {stat}")
        for network in tqdm(ismn_data.networks, desc=f"Networks for {stat}"):
            result = process_network(
                ismn_data, network, stat,
                base_output_dir, raw_ts_cache,
                overwrite=overwrite,
                log_list=log
            )
            result_counter[result] += 1

    return log, result_counter


In [7]:
if __name__ == '__main__':
    all_zip_files = []
    for root, dirs, files in os.walk(data_root):
        for file in files:
            if file.endswith('.zip'):
                all_zip_files.append(os.path.join(root, file))

    print(f"Total ISMN zip files found: {len(all_zip_files)}")

    overwrite_existing = False  # Set True to force overwrite
    total_log = []
    global_summary = {"success": 0, "skipped": 0, "failed": 0}

    for file_path in all_zip_files:
        file_log, summary = process_ismn_file(file_path, overwrite=overwrite_existing)
        total_log.extend(file_log)
        for k in global_summary:
            global_summary[k] += summary[k]

    # Write log to disk
    with open("ismn_run_log.txt", "w", encoding="utf-8") as f:
        for entry in total_log:
            f.write(entry + "\n")
        f.write("\n==== FINAL SUMMARY ====\n")
        for key, val in global_summary.items():
            f.write(f"{key.title()}: {val}\n")

    print("\n✅ Run complete.")
    print("Summary:")
    for key, val in global_summary.items():
        print(f"  {key.title()}: {val}")
    print("📝 Log saved to 'ismn_run_log.txt'")


Total ISMN zip files found: 2

📁 Processing file: Data_separate_files_header_20160614_20250614_11438_iyLN_20250614.zip
Using the existing ismn metadata in data\africa\python_metadata\Data_separate_files_header_20160614_20250614_11438_iyLN_20250614.csv to set up ISMN_Interface. 
If there are issues with the data reader, you can remove the metadata csv file to repeat metadata collection.

🔄 Stat: mean


Networks for mean: 100%|██████████| 3/3 [00:00<00:00, 1110.58it/s]


  ✔ Output exists for AMMA-CATCH - mean. Skipping.
  ✔ Output exists for SD_DEM - mean. Skipping.
  ✔ Output exists for TAHMO - mean. Skipping.

🔄 Stat: max


Networks for max: 100%|██████████| 3/3 [00:00<00:00, 775.10it/s]


  ✔ Output exists for AMMA-CATCH - max. Skipping.
  ✔ Output exists for SD_DEM - max. Skipping.
  ✔ Output exists for TAHMO - max. Skipping.

🔄 Stat: min


...

100%|██████████| 9/9 [00:00<00:00, 25.82it/s]


  ⏳ Processing WIT-Network with 9 sensors...


Networks for median: 100%|██████████| 11/11 [08:21<00:00, 45.62s/it]

File successfully written to data\asia\extracted_data\median\WIT-Network.csv
--------------------------------------------------
  ✓ Exported: data\asia\extracted_data\median\WIT-Network



✅ Run complete.
Summary:
  Success: 55
  Skipped: 15
  Failed: 0
📝 Log saved to 'ismn_run_log.txt'
